In [1]:
#Imports
import requests # Request module
import pandas as pd # Data Wrangling
import numpy as np # Data Wrangling
from bs4 import BeautifulSoup #Web sraping module
pd.set_option('display.max_columns', None)

In [2]:
year = [str(i) for i in range(2021, 2022)]

In [3]:
html_dfs = pd.read_html("https://www.pgatour.com/stats/stat.02671.{}.html".format(year))

In [4]:
df1 = html_dfs[1]

In [5]:
df1

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,EVENTS,POINTS,# OF WINS,# OF TOP-10S,POINTS BEHIND LEAD,RESET POINTS
0,1,10,Collin Morikawa,19,2136,2,8.0,0,NaN
1,2,5,Jordan Spieth,21,2072,1,9.0,64,NaN
2,3,1,Patrick Cantlay,20,2014,2,5.0,122,NaN
3,4,3,Jon Rahm,19,2003,1,12.0,133,NaN
4,5,2,Harris English,22,1899,2,7.0,237,NaN
...,...,...,...,...,...,...,...,...,...
245,246,245,Retief Goosen,1,3,0,NaN,2133,NaN
246,T247,T246,Arjun Atwal,8,3,0,NaN,2133,NaN
247,T247,T246,Zac Blair,4,3,0,NaN,2133,NaN
248,249,248,Jerry Kelly,3,3,0,NaN,2133,NaN


In [6]:
html_dfs2 = pd.read_html("https://www.pgatour.com/stats/stat.138.{}.html".format(year))

In [7]:
html_dfs2[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,TOP 10,1ST,2ND,3RD
0,1,1,Jon Rahm,12,1.0,1.0,1.0
1,2,2,Jordan Spieth,9,1.0,2.0,2.0
2,T3,T3,Scottie Scheffler,8,NaN,1.0,1.0
3,T3,T3,Brooks Koepka,8,1.0,2.0,NaN
4,T3,T3,Collin Morikawa,8,2.0,1.0,NaN
...,...,...,...,...,...,...,...
171,T122,T120,Peter Uihlein,1,NaN,NaN,1.0
172,T122,T120,Martin Laird,1,1.0,NaN,NaN
173,T122,T120,Rob Oppenheim,1,NaN,NaN,NaN
174,T122,T120,Michael Thompson,1,NaN,NaN,NaN


In [8]:
html_dfs3 = pd.read_html("https://www.pgatour.com/stats/stat.120.{}.html".format(year))

In [9]:
html_dfs3[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,AVG,TOTAL STROKES,TOTAL ADJUSTMENT,TOTAL ROUNDS
0,1,1,Jon Rahm,74,69.429,4514,- 1.096,65
1,2,2,Dustin Johnson,59,69.592,3918,- 20.872,56
2,3,4,Louis Oosthuizen,65,69.632,4061,- 22.332,58
3,4,8,Collin Morikawa,69,69.673,4448,11.064,64
4,5,6,Brooks Koepka,52,69.677,3631,- 7.815,52
...,...,...,...,...,...,...,...,...
200,201,201,Grayson Murray,52,73.105,3487,22.058,48
201,202,T204,Nick Watney,52,73.106,3470,39.098,48
202,203,203,Hunter Mahan,57,73.226,3998,29.425,55
203,204,T204,Martin Trainer,55,73.346,3860,27.332,53


In [10]:
html_dfs4 = pd.read_html("https://www.pgatour.com/stats/stat.101.{}.html".format(year))

In [11]:
html_dfs4[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,AVG.,TOTAL DISTANCE,TOTAL DRIVES
0,1,1,Bryson DeChambeau,67,320.9,41073,128
1,2,2,Rory McIlroy,61,317.7,36850,116
2,3,3,Cameron Champ,63,317.6,37480,118
3,4,4,Wyndham Clark,76,315.5,45438,144
4,5,5,Will Gordon,86,314.8,52888,168
...,...,...,...,...,...,...,...
203,204,204,Brian Stuard,96,277.6,52189,188
204,205,205,Tim Wilkinson,46,277.0,24373,88
205,206,207,Brendon Todd,85,274.1,42761,156
206,207,206,David Hearn,64,273.8,32860,120


In [12]:
html_dfs5 = pd.read_html("https://www.pgatour.com/stats/stat.102.{}.html".format(year))

In [13]:
html_dfs5[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,%,FAIRWAYS HIT,POSSIBLE FAIRWAYS
0,1,1,Brendon Todd,85,75.09,820,1092
1,2,2,Ryan Moore,42,73.29,428,584
2,3,3,Brian Stuard,96,73.21,959,1310
3,4,5,Ryan Armour,72,72.91,732,1004
4,5,4,Chez Reavie,76,72.63,751,1034
...,...,...,...,...,...,...,...
203,204,203,Jamie Lovemark,42,49.48,288,582
204,205,204,Michael Kim,65,49.18,418,850
205,206,206,Wyndham Clark,76,47.11,472,1002
206,207,208,Jimmy Walker,70,46.92,457,974


In [14]:
html_dfs6 = pd.read_html("https://www.pgatour.com/stats/stat.103.{}.html".format(year))

In [15]:
html_dfs6[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,%,GREENS HIT,# HOLES,RELATIVE/PAR
0,1,2,Collin Morikawa,69,72.05,830,1152,-0.32
1,2,1,Cameron Percy,73,71.90,893,1242,-0.25
2,3,3,Matthew NeSmith,84,71.75,1059,1476,-0.25
3,4,5,Jon Rahm,74,71.62,838,1170,-0.33
4,5,4,Emiliano Grillo,88,71.53,1133,1584,-0.29
...,...,...,...,...,...,...,...,...
203,204,203,Christiaan Bezuidenhout,47,60.23,477,792,-0.26
204,205,205,Henrik Stenson,47,59.56,461,774,-0.22
205,206,206,Jimmy Walker,70,57.22,721,1260,-0.27
206,207,207,Hunter Mahan,57,56.87,563,990,-0.28


In [16]:
html_dfs7 = pd.read_html("https://www.pgatour.com/stats/stat.02564.{}.html".format(year))

In [17]:
html_dfs7[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,AVERAGE,TOTAL SG:PUTTING,MEASURED ROUNDS
0,1,1,Louis Oosthuizen,65,1.062,46.731,44
1,2,2,Brendon Todd,85,0.792,49.884,63
2,3,7,J.T. Poston,79,0.763,54.140,71
3,4,3,Christiaan Bezuidenhout,47,0.760,24.306,32
4,5,4,Jason Kokrak,79,0.756,45.384,60
...,...,...,...,...,...,...,...
203,204,204,Tyler McCumber,60,-0.843,-38.783,46
204,205,205,Byeong Hun An,80,-0.889,-56.875,64
205,206,206,Cameron Champ,63,-0.965,-47.266,49
206,207,207,Grayson Murray,52,-1.289,-45.127,35


In [18]:
html_dfs8 = pd.read_html("https://www.pgatour.com/stats/stat.02674.{}.html".format(year))

In [19]:
html_dfs8[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,AVERAGE,SG:OTT,SG:APR,SG:ARG,MEASURED ROUNDS
0,1,1,Collin Morikawa,69,2.044,0.382,1.502,0.159,52
1,2,2,Jon Rahm,74,1.825,0.848,0.762,0.215,52
2,3,3,Brooks Koepka,52,1.528,0.616,0.738,0.173,38
3,4,4,Patrick Cantlay,69,1.505,0.550,0.537,0.418,52
4,5,5,Justin Thomas,71,1.469,0.213,0.853,0.402,52
...,...,...,...,...,...,...,...,...,...
203,204,203,Sebastian Cappelen,52,-1.526,-0.234,-0.711,-0.581,34
204,205,205,Sung Kang,80,-1.572,-0.627,-0.706,-0.239,67
205,206,206,Rhein Gibson,54,-1.694,-0.915,-0.612,-0.167,34
206,207,207,Martin Trainer,55,-2.158,-0.594,-1.491,-0.073,42


In [20]:
html_dfs9 = pd.read_html("https://www.pgatour.com/stats/stat.02675.{}.html".format(year))

In [21]:
html_dfs9[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,ROUNDS,AVERAGE,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,MEASURED ROUNDS
0,1,1,Jon Rahm,74,2.028,105.455,94.897,10.560,52
1,2,2,Louis Oosthuizen,65,1.911,84.099,37.370,46.731,44
2,3,3,Xander Schauffele,67,1.881,88.422,60.129,28.295,47
3,4,4,Brooks Koepka,52,1.876,71.305,58.047,13.259,38
4,5,5,Bryson DeChambeau,67,1.845,95.917,74.203,21.716,52
...,...,...,...,...,...,...,...,...,...
203,204,203,Sung Kang,80,-1.760,-117.902,-105.317,-12.580,67
204,205,204,Hunter Mahan,57,-1.802,-70.272,-93.170,22.900,39
205,206,205,Martin Trainer,55,-1.954,-82.078,-90.629,8.554,42
206,207,206,Grayson Murray,52,-2.094,-73.290,-28.161,-45.127,35


In [22]:
html_dfs10 = pd.read_html("https://www.pgatour.com/stats/stat.109.{}.html".format(year))

In [23]:
html_dfs10[1]

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,EVENTS,MONEY,YTD VICTORIES
0,1,9,Collin Morikawa,19,"$6,950,768",2.0
1,2,1,Jon Rahm,19,"$6,793,933",1.0
2,3,6,Jordan Spieth,21,"$6,214,964",1.0
3,4,2,Bryson DeChambeau,18,"$6,078,990",2.0
4,5,3,Justin Thomas,19,"$5,953,362",1.0
...,...,...,...,...,...,...
245,246,244,Fred Funk,1,"$9,000",NaN
246,T247,T245,Arjun Atwal,8,"$8,120",NaN
247,T247,T245,Zac Blair,4,"$8,120",NaN
248,249,248,Retief Goosen,1,"$6,810",NaN


In [24]:
df = html_dfs[1].merge(html_dfs2[1], on='PLAYER NAME').merge(html_dfs3[1], on= 'PLAYER NAME').merge(html_dfs4[1], on= 'PLAYER NAME').merge(html_dfs5[1], on= 'PLAYER NAME').merge(html_dfs6[1], on= 'PLAYER NAME').merge(html_dfs7[1], on= 'PLAYER NAME').merge(html_dfs8[1], on= 'PLAYER NAME').merge(html_dfs9[1], on= 'PLAYER NAME').merge(html_dfs10[1], on='PLAYER NAME')

In [25]:
df

,RANK THIS WEEK_x,RANK LAST WEEK_x,PLAYER NAME,EVENTS_x,POINTS,# OF WINS,# OF TOP-10S,POINTS BEHIND LEAD,RESET POINTS,RANK THIS WEEK_y,RANK LAST WEEK_y,TOP 10,1ST,2ND,3RD,RANK THIS WEEK_x,RANK LAST WEEK_x,ROUNDS_x,AVG,TOTAL STROKES,TOTAL ADJUSTMENT,TOTAL ROUNDS,RANK THIS WEEK_y,RANK LAST WEEK_y,ROUNDS_y,AVG.,TOTAL DISTANCE,TOTAL DRIVES,RANK THIS WEEK_x,RANK LAST WEEK_x,ROUNDS_x,%_x,FAIRWAYS HIT,POSSIBLE FAIRWAYS,RANK THIS WEEK_y,RANK LAST WEEK_y,ROUNDS_y,%_y,GREENS HIT,# HOLES,RELATIVE/PAR,RANK THIS WEEK_x,RANK LAST WEEK_x,ROUNDS_x,AVERAGE_x,TOTAL SG:PUTTING,MEASURED ROUNDS_x,RANK THIS WEEK_y,RANK LAST WEEK_y,ROUNDS_y,AVERAGE_y,SG:OTT,SG:APR,SG:ARG,MEASURED ROUNDS_y,RANK THIS WEEK_x,RANK LAST WEEK_x,ROUNDS,AVERAGE,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,MEASURED ROUNDS,RANK THIS WEEK_y,RANK LAST WEEK_y,EVENTS_y,MONEY,YTD VICTORIES
0,1,10,Collin Morikawa,19,2136,2,8.0,0,NaN,T3,T3,8,2.0,1.0,NaN,4,8,69,69.673,4448,11.064,64,114,T114,69,294.3,37674,128,13,11,69,69.20,620,896,1,2,69,72.05,830,1152,-0.32,170,172,69,-0.310,-16.140,52,1,1,69,2.044,0.382,1.502,0.159,52,7,7,69,1.733,90.134,106.277,-16.140,52,1,9,19,"$6,950,768",2.0
1,2,5,Jordan Spieth,21,2072,1,9.0,64,NaN,2,2,9,1.0,2.0,2.0,11,T20,76,69.909,5043,- 9.528,72,68,73,76,298.7,43006,144,187,188,76,54.29,544,1002,T113,121,76,65.28,846,1296,-0.34,32,34,76,0.436,25.277,58,20,19,76,1.014,-0.009,0.594,0.428,58,11,11,76,1.450,84.096,58.822,25.277,58,3,6,21,"$6,214,964",1.0
2,3,1,Patrick Cantlay,20,2014,2,5.0,122,NaN,T22,T21,5,2.0,1.0,1.0,14,10,69,69.975,4337,1.459,62,50,51,69,301.2,37350,124,118,111,69,59.91,520,868,19,22,69,69.27,773,1116,-0.28,54,53,69,0.278,14.436,52,4,4,69,1.505,0.550,0.537,0.418,52,6,6,69,1.783,92.716,78.283,14.436,52,7,4,20,"$5,620,030",2.0
3,4,3,Jon Rahm,19,2003,1,12.0,133,NaN,1,1,12,1.0,1.0,1.0,1,1,74,69.429,4514,- 1.096,65,20,21,74,307.7,40003,130,65,65,74,63.52,578,910,4,5,74,71.62,838,1170,-0.33,66,65,74,0.203,10.560,52,2,2,74,1.825,0.848,0.762,0.215,52,1,1,74,2.028,105.455,94.897,10.560,52,2,1,19,"$6,793,933",1.0
4,5,2,Harris English,22,1899,2,7.0,237,NaN,T6,T3,7,2.0,NaN,1.0,25,23,79,70.153,5311,20.655,76,73,69,79,298.4,45363,152,52,53,79,64.94,691,1064,85,T74,79,66.08,904,1368,-0.32,20,22,79,0.502,31.105,62,58,58,79,0.526,0.146,0.122,0.258,62,33,32,79,1.028,63.730,32.625,31.105,62,8,5,22,"$5,563,406",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,175,175,Jimmy Walker,24,162,0,1.0,1974,NaN,T122,T120,1,NaN,NaN,NaN,191,194,70,72.294,5078,- 17.411,70,88,T86,70,296.8,41550,140,207,208,70,46.92,457,974,206,206,70,57.22,721,1260,-0.27,47,47,70,0.312,18.380,59,198,198,70,-1.214,-0.700,-0.364,-0.150,59,188,188,70,-0.902,-53.226,-71.600,18.380,59,159,158,24,"$519,717",NaN
155,177,180,Mark Anderson,18,160,0,1.0,1976,NaN,T122,T120,1,NaN,NaN,NaN,165,170,52,71.752,3541,46.604,50,191,191,52,284.6,28462,100,T57,69,52,64.14,449,700,35,T40,52,68.22,614,900,-0.26,T127,123,52,-0.067,-1.862,28,108,135,52,0.070,-0.035,-0.143,0.248,28,T122,146,52,0.004,0.098,1.962,-1.862,28,177,181,18,"$338,181",NaN
156,179,179,"Ted Potter, Jr.",23,157,0,1.0,1979,NaN,T122,T120,1,NaN,NaN,NaN,184,187,64,72.113,4582,33.235,64,175,T174,64,287.1,36751,128,166,159,64,56.40,502,890,190,T188,64,61.89,713,1152,-0.28,T186,187,64,-0.461,-24.882,54,181,182,64,-0.637,-0.538,-0.074,-0.026,54,193,195,64,-1.098,-59.274,-34.389,-24.882,54,174,174,23,"$355,877",NaN
157,185,185,Grayson Murray,21,122,0,1.0,2014,NaN,T122,T120,1,NaN,NaN,1.0,201,201,52,73.105,3487,22.058,48,28,29,52,305.2,29297,96,200,199,52,50.60,338,668,T177,178,52,62.62,541,864,-0.25,207,207,52,-1.289,-45.127,35,191,186,52,-0.805,-0.050,-0.290,-0.466,35,207,206,52,-2.094,-73.290,-28.161,-45.127,35,185,185,21,"$255,130",NaN


In [26]:
df.columns=df.columns.str.replace(' ', '_')
df.columns = map(str.lower, df.columns)

In [27]:
df

,rank this_week_x,rank_last_week_x,player_name,events_x,points,#_of_wins,#_of_top-10s,points_behind_lead,reset_points,rank this_week_y,rank_last_week_y,top_10,1st,2nd,3rd,rank this_week_x,rank_last_week_x,rounds_x,avg,total_strokes,total_adjustment,total_rounds,rank this_week_y,rank_last_week_y,rounds_y,avg.,total_distance,total_drives,rank this_week_x,rank_last_week_x,rounds_x,%_x,fairways_hit,possible_fairways,rank this_week_y,rank_last_week_y,rounds_y,%_y,greens_hit,#_holes,relative/par,rank this_week_x,rank_last_week_x,rounds_x,average_x,total_sg:putting,measured_rounds_x,rank this_week_y,rank_last_week_y,rounds_y,average_y,sg:ott,sg:apr,sg:arg,measured_rounds_y,rank this_week_x,rank_last_week_x,rounds,average,total_sg:t,total_sg:t2g,total_sg:p,measured_rounds,rank this_week_y,rank_last_week_y,events_y,money,ytd_victories
0,1,10,Collin Morikawa,19,2136,2,8.0,0,NaN,T3,T3,8,2.0,1.0,NaN,4,8,69,69.673,4448,11.064,64,114,T114,69,294.3,37674,128,13,11,69,69.20,620,896,1,2,69,72.05,830,1152,-0.32,170,172,69,-0.310,-16.140,52,1,1,69,2.044,0.382,1.502,0.159,52,7,7,69,1.733,90.134,106.277,-16.140,52,1,9,19,"$6,950,768",2.0
1,2,5,Jordan Spieth,21,2072,1,9.0,64,NaN,2,2,9,1.0,2.0,2.0,11,T20,76,69.909,5043,- 9.528,72,68,73,76,298.7,43006,144,187,188,76,54.29,544,1002,T113,121,76,65.28,846,1296,-0.34,32,34,76,0.436,25.277,58,20,19,76,1.014,-0.009,0.594,0.428,58,11,11,76,1.450,84.096,58.822,25.277,58,3,6,21,"$6,214,964",1.0
2,3,1,Patrick Cantlay,20,2014,2,5.0,122,NaN,T22,T21,5,2.0,1.0,1.0,14,10,69,69.975,4337,1.459,62,50,51,69,301.2,37350,124,118,111,69,59.91,520,868,19,22,69,69.27,773,1116,-0.28,54,53,69,0.278,14.436,52,4,4,69,1.505,0.550,0.537,0.418,52,6,6,69,1.783,92.716,78.283,14.436,52,7,4,20,"$5,620,030",2.0
3,4,3,Jon Rahm,19,2003,1,12.0,133,NaN,1,1,12,1.0,1.0,1.0,1,1,74,69.429,4514,- 1.096,65,20,21,74,307.7,40003,130,65,65,74,63.52,578,910,4,5,74,71.62,838,1170,-0.33,66,65,74,0.203,10.560,52,2,2,74,1.825,0.848,0.762,0.215,52,1,1,74,2.028,105.455,94.897,10.560,52,2,1,19,"$6,793,933",1.0
4,5,2,Harris English,22,1899,2,7.0,237,NaN,T6,T3,7,2.0,NaN,1.0,25,23,79,70.153,5311,20.655,76,73,69,79,298.4,45363,152,52,53,79,64.94,691,1064,85,T74,79,66.08,904,1368,-0.32,20,22,79,0.502,31.105,62,58,58,79,0.526,0.146,0.122,0.258,62,33,32,79,1.028,63.730,32.625,31.105,62,8,5,22,"$5,563,406",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,175,175,Jimmy Walker,24,162,0,1.0,1974,NaN,T122,T120,1,NaN,NaN,NaN,191,194,70,72.294,5078,- 17.411,70,88,T86,70,296.8,41550,140,207,208,70,46.92,457,974,206,206,70,57.22,721,1260,-0.27,47,47,70,0.312,18.380,59,198,198,70,-1.214,-0.700,-0.364,-0.150,59,188,188,70,-0.902,-53.226,-71.600,18.380,59,159,158,24,"$519,717",NaN
155,177,180,Mark Anderson,18,160,0,1.0,1976,NaN,T122,T120,1,NaN,NaN,NaN,165,170,52,71.752,3541,46.604,50,191,191,52,284.6,28462,100,T57,69,52,64.14,449,700,35,T40,52,68.22,614,900,-0.26,T127,123,52,-0.067,-1.862,28,108,135,52,0.070,-0.035,-0.143,0.248,28,T122,146,52,0.004,0.098,1.962,-1.862,28,177,181,18,"$338,181",NaN
156,179,179,"Ted Potter, Jr.",23,157,0,1.0,1979,NaN,T122,T120,1,NaN,NaN,NaN,184,187,64,72.113,4582,33.235,64,175,T174,64,287.1,36751,128,166,159,64,56.40,502,890,190,T188,64,61.89,713,1152,-0.28,T186,187,64,-0.461,-24.882,54,181,182,64,-0.637,-0.538,-0.074,-0.026,54,193,195,64,-1.098,-59.274,-34.389,-24.882,54,174,174,23,"$355,877",NaN
157,185,185,Grayson Murray,21,122,0,1.0,2014,NaN,T122,T120,1,NaN,NaN,1.0,201,201,52,73.105,3487,22.058,48,28,29,52,305.2,29297,96,200,199,52,50.60,338,668,T177,178,52,62.62,541,864,-0.25,207,207,52,-1.289,-45.127,35,191,186,52,-0.805,-0.050,-0.290,-0.466,35,207,206,52,-2.094,-73.290,-28.161,-45.127,35,185,185,21,"$255,130",NaN


In [28]:
df.columns

Index(['rank this_week_x', 'rank_last_week_x', 'player_name', 'events_x',
       'points', '#_of_wins', '#_of_top-10s', 'points_behind_lead',
       'reset_points', 'rank this_week_y', 'rank_last_week_y', 'top_10', '1st',
       '2nd', '3rd', 'rank this_week_x', 'rank_last_week_x', 'rounds_x', 'avg',
       'total_strokes', 'total_adjustment', 'total_rounds', 'rank this_week_y',
       'rank_last_week_y', 'rounds_y', 'avg.', 'total_distance',
       'total_drives', 'rank this_week_x', 'rank_last_week_x', 'rounds_x',
       '%_x', 'fairways_hit', 'possible_fairways', 'rank this_week_y',
       'rank_last_week_y', 'rounds_y', '%_y', 'greens_hit', '#_holes',
       'relative/par', 'rank this_week_x', 'rank_last_week_x', 'rounds_x',
       'average_x', 'total_sg:putting', 'measured_rounds_x',
       'rank this_week_y', 'rank_last_week_y', 'rounds_y', 'average_y',
       'sg:ott', 'sg:apr', 'sg:arg', 'measured_rounds_y', 'rank this_week_x',
       'rank_last_week_x', 'rounds', 'average', 't

In [29]:
df = df.drop(df.columns[[0,1,3,7,8,9,10,12,13,14,15,16,17,22,23,24,28,29,34,35,41,42,46,47,48,54,55,56,62]],axis=1)

In [30]:
df

,player_name,points,#_of_wins,#_of_top-10s,top_10,avg,total_strokes,total_adjustment,total_rounds,avg.,total_distance,total_drives,%_x,fairways_hit,possible_fairways,%_y,greens_hit,#_holes,relative/par,average_x,total_sg:putting,average_y,sg:ott,sg:apr,sg:arg,rounds,average,total_sg:t,total_sg:t2g,total_sg:p,events_y,money,ytd_victories
0,Collin Morikawa,2136,2,8.0,8,69.673,4448,11.064,64,294.3,37674,128,69.20,620,896,72.05,830,1152,-0.32,-0.310,-16.140,2.044,0.382,1.502,0.159,69,1.733,90.134,106.277,-16.140,19,"$6,950,768",2.0
1,Jordan Spieth,2072,1,9.0,9,69.909,5043,- 9.528,72,298.7,43006,144,54.29,544,1002,65.28,846,1296,-0.34,0.436,25.277,1.014,-0.009,0.594,0.428,76,1.450,84.096,58.822,25.277,21,"$6,214,964",1.0
2,Patrick Cantlay,2014,2,5.0,5,69.975,4337,1.459,62,301.2,37350,124,59.91,520,868,69.27,773,1116,-0.28,0.278,14.436,1.505,0.550,0.537,0.418,69,1.783,92.716,78.283,14.436,20,"$5,620,030",2.0
3,Jon Rahm,2003,1,12.0,12,69.429,4514,- 1.096,65,307.7,40003,130,63.52,578,910,71.62,838,1170,-0.33,0.203,10.560,1.825,0.848,0.762,0.215,74,2.028,105.455,94.897,10.560,19,"$6,793,933",1.0
4,Harris English,1899,2,7.0,7,70.153,5311,20.655,76,298.4,45363,152,64.94,691,1064,66.08,904,1368,-0.32,0.502,31.105,0.526,0.146,0.122,0.258,79,1.028,63.730,32.625,31.105,22,"$5,563,406",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Jimmy Walker,162,0,1.0,1,72.294,5078,- 17.411,70,296.8,41550,140,46.92,457,974,57.22,721,1260,-0.27,0.312,18.380,-1.214,-0.700,-0.364,-0.150,70,-0.902,-53.226,-71.600,18.380,24,"$519,717",NaN
155,Mark Anderson,160,0,1.0,1,71.752,3541,46.604,50,284.6,28462,100,64.14,449,700,68.22,614,900,-0.26,-0.067,-1.862,0.070,-0.035,-0.143,0.248,52,0.004,0.098,1.962,-1.862,18,"$338,181",NaN
156,"Ted Potter, Jr.",157,0,1.0,1,72.113,4582,33.235,64,287.1,36751,128,56.40,502,890,61.89,713,1152,-0.28,-0.461,-24.882,-0.637,-0.538,-0.074,-0.026,64,-1.098,-59.274,-34.389,-24.882,23,"$355,877",NaN
157,Grayson Murray,122,0,1.0,1,73.105,3487,22.058,48,305.2,29297,96,50.60,338,668,62.62,541,864,-0.25,-1.289,-45.127,-0.805,-0.050,-0.290,-0.466,52,-2.094,-73.290,-28.161,-45.127,21,"$255,130",NaN


In [31]:
df = df.drop(df.columns[[3,6,7,8,9,18,19,21,26]],axis=1)
df

,player_name,points,#_of_wins,top_10,avg,total_distance,total_drives,%_x,fairways_hit,possible_fairways,%_y,greens_hit,#_holes,total_sg:putting,sg:ott,sg:apr,sg:arg,rounds,total_sg:t,total_sg:t2g,total_sg:p,events_y,money,ytd_victories
0,Collin Morikawa,2136,2,8,69.673,37674,128,69.20,620,896,72.05,830,1152,-16.140,0.382,1.502,0.159,69,90.134,106.277,-16.140,19,"$6,950,768",2.0
1,Jordan Spieth,2072,1,9,69.909,43006,144,54.29,544,1002,65.28,846,1296,25.277,-0.009,0.594,0.428,76,84.096,58.822,25.277,21,"$6,214,964",1.0
2,Patrick Cantlay,2014,2,5,69.975,37350,124,59.91,520,868,69.27,773,1116,14.436,0.550,0.537,0.418,69,92.716,78.283,14.436,20,"$5,620,030",2.0
3,Jon Rahm,2003,1,12,69.429,40003,130,63.52,578,910,71.62,838,1170,10.560,0.848,0.762,0.215,74,105.455,94.897,10.560,19,"$6,793,933",1.0
4,Harris English,1899,2,7,70.153,45363,152,64.94,691,1064,66.08,904,1368,31.105,0.146,0.122,0.258,79,63.730,32.625,31.105,22,"$5,563,406",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Jimmy Walker,162,0,1,72.294,41550,140,46.92,457,974,57.22,721,1260,18.380,-0.700,-0.364,-0.150,70,-53.226,-71.600,18.380,24,"$519,717",NaN
155,Mark Anderson,160,0,1,71.752,28462,100,64.14,449,700,68.22,614,900,-1.862,-0.035,-0.143,0.248,52,0.098,1.962,-1.862,18,"$338,181",NaN
156,"Ted Potter, Jr.",157,0,1,72.113,36751,128,56.40,502,890,61.89,713,1152,-24.882,-0.538,-0.074,-0.026,64,-59.274,-34.389,-24.882,23,"$355,877",NaN
157,Grayson Murray,122,0,1,73.105,29297,96,50.60,338,668,62.62,541,864,-45.127,-0.050,-0.290,-0.466,52,-73.290,-28.161,-45.127,21,"$255,130",NaN


In [32]:
df = df.rename(columns={'#_of_wins': 'wins', 'avg': 'avg_score', '%_x':'fairway_%','%_y':'%_gir',})

In [33]:
# Replace NaN with 0 in Top 10 
df['top_10'].fillna(0, inplace=True)
df['top_10'] = df['top_10'].astype(int)





# Change Rounds to int
df['rounds'] = df['rounds'].astype(int)


df['points'] = df['points'].astype(int)

df.columns = map(str.lower, df.columns)
df["money"] = df["money"].str.replace("$", "")
df["money"] = df["money"].str.replace(",", "")
df["money"] = df["money"].str.replace(".", "")
df["money"] = df["money"].astype('float64')
#df["player_name"] = df["player_name"].str.replace(' ', '_')
df.columns=df.columns.str.replace(' ', '_')
df["player_name"] = df["player_name"].str.replace(' ', '_')
df

,player_name,points,wins,top_10,avg_score,total_distance,total_drives,fairway_%,fairways_hit,possible_fairways,%_gir,greens_hit,#_holes,total_sg:putting,sg:ott,sg:apr,sg:arg,rounds,total_sg:t,total_sg:t2g,total_sg:p,events_y,money,ytd_victories
0,Collin_Morikawa,2136,2,8,69.673,37674,128,69.20,620,896,72.05,830,1152,-16.140,0.382,1.502,0.159,69,90.134,106.277,-16.140,19,6950768.0,2.0
1,Jordan_Spieth,2072,1,9,69.909,43006,144,54.29,544,1002,65.28,846,1296,25.277,-0.009,0.594,0.428,76,84.096,58.822,25.277,21,6214964.0,1.0
2,Patrick_Cantlay,2014,2,5,69.975,37350,124,59.91,520,868,69.27,773,1116,14.436,0.550,0.537,0.418,69,92.716,78.283,14.436,20,5620030.0,2.0
3,Jon_Rahm,2003,1,12,69.429,40003,130,63.52,578,910,71.62,838,1170,10.560,0.848,0.762,0.215,74,105.455,94.897,10.560,19,6793933.0,1.0
4,Harris_English,1899,2,7,70.153,45363,152,64.94,691,1064,66.08,904,1368,31.105,0.146,0.122,0.258,79,63.730,32.625,31.105,22,5563406.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Jimmy_Walker,162,0,1,72.294,41550,140,46.92,457,974,57.22,721,1260,18.380,-0.700,-0.364,-0.150,70,-53.226,-71.600,18.380,24,519717.0,NaN
155,Mark_Anderson,160,0,1,71.752,28462,100,64.14,449,700,68.22,614,900,-1.862,-0.035,-0.143,0.248,52,0.098,1.962,-1.862,18,338181.0,NaN
156,"Ted_Potter,_Jr.",157,0,1,72.113,36751,128,56.40,502,890,61.89,713,1152,-24.882,-0.538,-0.074,-0.026,64,-59.274,-34.389,-24.882,23,355877.0,NaN
157,Grayson_Murray,122,0,1,73.105,29297,96,50.60,338,668,62.62,541,864,-45.127,-0.050,-0.290,-0.466,52,-73.290,-28.161,-45.127,21,255130.0,NaN


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        159 non-null    object 
 1   points             159 non-null    int64  
 2   wins               159 non-null    int64  
 3   top_10             159 non-null    int64  
 4   avg_score          159 non-null    float64
 5   total_distance     159 non-null    int64  
 6   total_drives       159 non-null    int64  
 7   fairway_%          159 non-null    float64
 8   fairways_hit       159 non-null    int64  
 9   possible_fairways  159 non-null    int64  
 10  %_gir              159 non-null    float64
 11  greens_hit         159 non-null    int64  
 12  #_holes            159 non-null    int64  
 13  total_sg:putting   159 non-null    float64
 14  sg:ott             159 non-null    float64
 15  sg:apr             159 non-null    float64
 16  sg:arg             159 non

In [35]:
df.to_csv(path_or_buf='../../../data/pgaTourData2021_clean.csv', index = False)